In [1]:
import keras

Using TensorFlow backend.
/home/jared/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Data Prep

In [2]:
%cd data

data_dir = %pwd
train_dir = data_dir + '/train/'
valid_dir = data_dir + '/valid/'
test_dir = data_dir + '/test/'
sample_dir = data_dir + '/sample/'

/home/jared/courses/deeplearning2/Kaggle/dogscats/data


In [3]:
import os
from glob import glob
import numpy as np
from shutil import copyfile

num_valid = 1000
num_sample = 64

classes = ['cat', 'dog']


In [28]:
for cls in classes:
    os.mkdir(train_dir + cls, 775)
    os.mkdir(valid_dir + cls, 775)
    os.mkdir(sample_dir + 'train/' + cls, 775)
    os.mkdir(sample_dir + 'valid/' + cls, 775)    

In [29]:

for cls in classes:
    files = glob(train_dir + cls + '*.jpg')
    np.random.shuffle(files)

    for file in files[:num_sample]:
        copyfile(file, sample_dir + 'train/' + cls + '/' + os.path.basename(file))
        
    for file in files[num_sample:num_sample+num_sample]:
        copyfile(file, sample_dir + 'valid/' + cls + '/' + os.path.basename(file))
    
    for file in files[:num_valid]:
        os.rename(file, valid_dir + cls + '/' + os.path.basename(file))
        
    for file in files[num_valid:]:
        os.rename(file, train_dir + cls + '/' + os.path.basename(file))
    


# Simple Model

In [4]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Flatten, Convolution2D, MaxPooling2D, Input
from keras.optimizers import Adam

#base_dir = sample_dir
path = data_dir + '/'
batch_size = 64
image_size = (224,224)
image_shape = (*image_size, 3)

In [5]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator()
val_gen = ImageDataGenerator()

batches = gen.flow_from_directory(path+'train/', target_size=image_size, batch_size=batch_size, shuffle=True)
val_batches = val_gen.flow_from_directory(path+'valid/', target_size=image_size, batch_size=batch_size, shuffle=True)

train_steps = batches.n//batch_size
val_steps = val_batches.n//batch_size

if (val_steps == 0):
    val_steps = 1

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [104]:
model = Sequential([
    BatchNormalization(input_shape=image_shape)
    , Convolution2D(64, (3,3), activation='relu')
    , MaxPooling2D((2,2))
    , Convolution2D(64, (3,3), activation='relu')
    , MaxPooling2D((2,2))
    , Flatten()
    , Dense(124, activation='relu')
    , Dense(2, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 126, 126, 64)      1792      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 61, 61, 64)        36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 57600)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 124)               7142524   
__________

In [96]:
model.fit_generator(batches, train_steps, epochs=1, validation_data=val_batches, validation_steps=val_steps)

Epoch 1/1
359/359 [==============================] - 75s - loss: 0.6704 - acc: 0.6494 - val_loss: 0.5757 - val_acc: 0.6699


# Transfer Learning

In [6]:
def get_batches(dirname, gen=ImageDataGenerator(), shuffle=True, batch_size=4, class_mode='categorical',
                target_size=(224,224)):
    return gen.flow_from_directory(dirname, target_size=target_size,
                                    class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [7]:
def get_data(path, target_size=(224,224)):
    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None, target_size=target_size)
    return np.concatenate([batches.next() for i in range((batches.n//batch_size) + 1)])

In [8]:
gen = ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True)
val_gen = ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True)

gen.fit(get_data(path+'train/'))
val_gen.fit(get_data(path+'train/'))

Found 23000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.


In [9]:
batches = gen.flow_from_directory(path+'train/', target_size=image_size, batch_size=batch_size, shuffle=False)
val_batches = val_gen.flow_from_directory(path+'valid/', target_size=image_size, batch_size=batch_size, shuffle=False)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [9]:
from keras.applications.vgg16 import VGG16

vgg = VGG16(include_top=False, input_shape=image_shape, pooling='avg')
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [17]:
X_train = vgg.predict_generator(batches, train_steps+1)
y_train = keras.utils.np_utils.to_categorical(batches.classes)

In [18]:
X_valid = vgg.predict_generator(val_batches, val_steps+1)
y_valid = val_batches.classes

In [10]:
import bcolz

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]

In [20]:
save_array(path + 'train_vgg_average_pool_features_normalized.bc', X_train)
save_array(path + 'valid_vgg_average_pool_features_normalized.bc', X_valid)

print(X_train.shape, X_valid.shape)

(23000, 512) (2000, 512)


In [11]:
X_train = load_array(path + 'train_vgg_average_pool_features_normalized.bc')
X_valid = load_array(path + 'valid_vgg_average_pool_features_normalized.bc')

print(X_train.shape, X_valid.shape)

(23000, 512) (2000, 512)


In [12]:
y_train = keras.utils.np_utils.to_categorical(batches.classes)
y_valid = keras.utils.np_utils.to_categorical(val_batches.classes)

In [13]:
model = Sequential([
    BatchNormalization(input_shape=vgg.output_shape[1:])
    , Dense(512, activation='relu')
    , BatchNormalization()
    , Dropout(.5)
    , Dense(2, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 267,778
Trainable params: 265,730
Non-trainable params: 2,048
_________________________________________________________________


In [29]:
model.fit(X_train, y_train, epochs=4, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/4
23000/23000 [==============================] - 5s - loss: 0.1695 - acc: 0.9393 - val_loss: 0.0902 - val_acc: 0.9655
Epoch 2/4
23000/23000 [==============================] - 5s - loss: 0.1135 - acc: 0.9561 - val_loss: 0.0817 - val_acc: 0.9670
Epoch 3/4
23000/23000 [==============================] - 5s - loss: 0.0971 - acc: 0.9602 - val_loss: 0.0878 - val_acc: 0.9675
Epoch 4/4
23000/23000 [==============================] - 5s - loss: 0.0973 - acc: 0.9621 - val_loss: 0.0789 - val_acc: 0.9735


In [25]:
model.optimizer.lr = .0001
model.fit(X_train, y_train, epochs=8, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/8
23000/23000 [==============================] - 5s - loss: 0.0844 - acc: 0.9683 - val_loss: 0.0793 - val_acc: 0.9715
Epoch 2/8
23000/23000 [==============================] - 5s - loss: 0.0796 - acc: 0.9682 - val_loss: 0.0695 - val_acc: 0.9755
Epoch 3/8
23000/23000 [==============================] - 5s - loss: 0.0794 - acc: 0.9687 - val_loss: 0.0806 - val_acc: 0.9760
Epoch 4/8
23000/23000 [==============================] - 5s - loss: 0.0793 - acc: 0.9697 - val_loss: 0.0855 - val_acc: 0.9655
Epoch 5/8
23000/23000 [==============================] - 5s - loss: 0.0638 - acc: 0.9749 - val_loss: 0.0836 - val_acc: 0.9710
Epoch 6/8
23000/23000 [==============================] - 5s - loss: 0.0642 - acc: 0.9755 - val_loss: 0.0748 - val_acc: 0.9740
Epoch 7/8
23000/23000 [==============================] - 5s - loss: 0.0632 - acc: 0.9768 - val_loss: 0.0775 - val_acc: 0.9715
Epoch 8/8
23000/23000 [==============================] - 5s - loss: 0

In [26]:
model.optimizer.lr = .00001
model.fit(X_train, y_train, epochs=12, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/12
23000/23000 [==============================] - 5s - loss: 0.0536 - acc: 0.9800 - val_loss: 0.0810 - val_acc: 0.9735
Epoch 2/12
23000/23000 [==============================] - 5s - loss: 0.0493 - acc: 0.9804 - val_loss: 0.0891 - val_acc: 0.9740
Epoch 3/12
23000/23000 [==============================] - 5s - loss: 0.0497 - acc: 0.9820 - val_loss: 0.0889 - val_acc: 0.9695
Epoch 4/12
23000/23000 [==============================] - 5s - loss: 0.0492 - acc: 0.9817 - val_loss: 0.0928 - val_acc: 0.9685
Epoch 5/12
23000/23000 [==============================] - 5s - loss: 0.0485 - acc: 0.9818 - val_loss: 0.0935 - val_acc: 0.9710
Epoch 6/12
23000/23000 [==============================] - 5s - loss: 0.0424 - acc: 0.9838 - val_loss: 0.0971 - val_acc: 0.9725
Epoch 7/12
23000/23000 [==============================] - 5s - loss: 0.0393 - acc: 0.9859 - val_loss: 0.0953 - val_acc: 0.9705
Epoch 8/12
23000/23000 [==============================] - 5s -

In [27]:
model.optimizer.lr = .000001
model.fit(X_train, y_train, epochs=16, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/16
23000/23000 [==============================] - 5s - loss: 0.0329 - acc: 0.9885 - val_loss: 0.1090 - val_acc: 0.9685
Epoch 2/16
23000/23000 [==============================] - 5s - loss: 0.0391 - acc: 0.9862 - val_loss: 0.1042 - val_acc: 0.9710
Epoch 3/16
23000/23000 [==============================] - 5s - loss: 0.0301 - acc: 0.9892 - val_loss: 0.1051 - val_acc: 0.9710
Epoch 4/16
23000/23000 [==============================] - 5s - loss: 0.0318 - acc: 0.9887 - val_loss: 0.1068 - val_acc: 0.9725
Epoch 5/16
23000/23000 [==============================] - 5s - loss: 0.0287 - acc: 0.9901 - val_loss: 0.1088 - val_acc: 0.9745
Epoch 6/16
23000/23000 [==============================] - 5s - loss: 0.0295 - acc: 0.9891 - val_loss: 0.1061 - val_acc: 0.9740
Epoch 7/16
23000/23000 [==============================] - 5s - loss: 0.0282 - acc: 0.9901 - val_loss: 0.1011 - val_acc: 0.9710
Epoch 8/16
23000/23000 [==============================] - 5s -

# Data Augmentation

In [15]:
def get_augmented_images(path, image_size, batch_size):
    gen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=90.,
        zoom_range=2.,
        fill_mode='nearest',
        horizontal_flip=True,
        vertical_flip=True)
    
    gen.fit(get_data(path))

    return gen.flow_from_directory(path, target_size=image_size, batch_size=batch_size, shuffle=False)

In [16]:
X_train_aug_1 = vgg.predict_generator(get_augmented_images(path+'train/', image_size, batch_size), train_steps+1)
X_train_aug_2 = vgg.predict_generator(get_augmented_images(path+'train/', image_size, batch_size), train_steps+1)
X_train_aug_3 = vgg.predict_generator(get_augmented_images(path+'train/', image_size, batch_size), train_steps+1)

Found 23000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.


In [17]:
save_array(path + 'train_vgg_average_pool_features_augmented_normalized_1.bc', X_train_aug_1)
save_array(path + 'train_vgg_average_pool_features_augmented_normalized_2.bc', X_train_aug_2)
save_array(path + 'train_vgg_average_pool_features_augmented_normalized_3.bc', X_train_aug_3)

In [18]:
y_train = keras.utils.np_utils.to_categorical(batches.classes)

In [19]:
X_train = load_array(path + 'train_vgg_average_pool_features.bc')
X_train_aug_1 = load_array(path + 'train_vgg_average_pool_features_augmented_normalized_1.bc')
X_train_aug_2 = load_array(path + 'train_vgg_average_pool_features_augmented_normalized_2.bc')
X_train_aug_3 = load_array(path + 'train_vgg_average_pool_features_augmented_normalized_3.bc')

print(X_train.shape, X_train_aug_1.shape)

(23000, 512) (23000, 512)


In [20]:
X_valid = load_array(path + 'valid_vgg_average_pool_features_normalized.bc')
y_valid = keras.utils.np_utils.to_categorical(val_batches.classes)

In [14]:
model = Sequential([
    BatchNormalization(input_shape=vgg.output_shape[1:])
    , Dropout(.5)
    , Dense(512, activation='relu')
    , BatchNormalization()
    , Dropout(.5)
    , Dense(1024, activation='relu')
    , BatchNormalization()
    , Dropout(.5)
    , Dense(2, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_4 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              525312    
_________________________________________________________________
batch_normalization_5 (Batch (None, 1024)              4096      
__________

In [22]:
model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))
model.fit(X_train_aug_1, y_train, epochs=1, validation_data=(X_valid, y_valid))
model.fit(X_train_aug_2, y_train, epochs=1, validation_data=(X_valid, y_valid))
model.fit(X_train_aug_3, y_train, epochs=1, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 7s - loss: 0.2579 - acc: 0.9115 - val_loss: 0.8421 - val_acc: 0.5000
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.6471 - acc: 0.6567 - val_loss: 0.1451 - val_acc: 0.9430
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.6022 - acc: 0.6773 - val_loss: 0.1652 - val_acc: 0.9330
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5826 - acc: 0.6921 - val_loss: 0.1485 - val_acc: 0.9490


In [23]:
model.optimizer.lr = .0001
for _ in range(2):
    model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_1, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_2, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_3, y_train, epochs=1, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.1645 - acc: 0.9316 - val_loss: 0.6391 - val_acc: 0.5005
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5943 - acc: 0.6830 - val_loss: 0.1807 - val_acc: 0.9380
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5701 - acc: 0.6978 - val_loss: 0.1185 - val_acc: 0.9530
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5674 - acc: 0.6993 - val_loss: 0.1451 - val_acc: 0.9405
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.1572 - acc: 0.9363 - val_loss: 0.6776 - val_acc: 0.5000
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5782 -

In [24]:
model.optimizer.lr = .00001
for _ in range(3):
    model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_1, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_2, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_3, y_train, epochs=1, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.1560 - acc: 0.9387 - val_loss: 0.6435 - val_acc: 0.5000
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5622 - acc: 0.7022 - val_loss: 0.1723 - val_acc: 0.9430
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5521 - acc: 0.7046 - val_loss: 0.1229 - val_acc: 0.9575
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5474 - acc: 0.7123 - val_loss: 0.1465 - val_acc: 0.9490
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.1500 - acc: 0.9392 - val_loss: 0.7095 - val_acc: 0.5000
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5581 -

In [25]:
model.optimizer.lr = .000001
for _ in range(5):
    model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_1, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_2, y_train, epochs=1, validation_data=(X_valid, y_valid))
    model.fit(X_train_aug_3, y_train, epochs=1, validation_data=(X_valid, y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.1526 - acc: 0.9395 - val_loss: 0.6192 - val_acc: 0.5795
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 7s - loss: 0.5367 - acc: 0.7171 - val_loss: 0.1951 - val_acc: 0.9455
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5316 - acc: 0.7221 - val_loss: 0.1190 - val_acc: 0.9555
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5232 - acc: 0.7257 - val_loss: 0.1164 - val_acc: 0.9595
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.1438 - acc: 0.9433 - val_loss: 0.6778 - val_acc: 0.5000
Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 6s - loss: 0.5358 -

# Ensamble

In [28]:
from keras.models import load_model

def get_learner():
    model = Sequential([
        BatchNormalization(input_shape=vgg.output_shape[1:])
        , Dropout(.5)
        , Dense(512, activation='relu')
        , BatchNormalization()
        , Dropout(.5)
        , Dense(1024, activation='relu')
        , BatchNormalization()
        , Dropout(.5)
        , Dense(2, activation='softmax')
    ])

    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def get_data():
    X_train = load_array(path + 'train_vgg_average_pool_features_normalized.bc')
    X_valid = load_array(path + 'valid_vgg_average_pool_features_normalized.bc')

#     idxs = [i for i in range(len(X_train))]
#     np.random.shuffle(idxs)
#     idxs = idxs[:len(X_train)//2]
        
    y_train = keras.utils.np_utils.to_categorical(batches.classes)
    y_valid = keras.utils.np_utils.to_categorical(val_batches.classes)
    
#     X_train = [X_train[i] for i in idxs]
#     y_train = [y_train[i] for i in idxs]
    
    
    return X_train, y_train, X_valid, y_valid
    
def train_learner(model, X_train, y_train, X_valid, y_valid, num_epochs=4, learning_rate=.001):
    print("Training model for {} epochs with learning rate {}".format(num_epochs, learning_rate))
    
    model.optimizer.lr = learning_rate
    
    model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_valid, y_valid), verbose=0)
    
def train_one_learner():
    model = get_learner()
    X_train, y_train, X_valid, y_valid = get_data()
    
    train_learner(model, X_train, y_train, X_valid, y_valid, num_epochs=4, learning_rate=.001)
    train_learner(model, X_train, y_train, X_valid, y_valid, num_epochs=8, learning_rate=.0001)
    train_learner(model, X_train, y_train, X_valid, y_valid, num_epochs=12, learning_rate=.00001)
    #train_learner(model, X_train, y_train, X_valid, y_valid, num_epochs=16, learning_rate=.000001)
    
    return model    

In [30]:
model = get_learner()
X_train, y_train, X_valid, y_valid = get_data()

model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))

'/home/jared/courses/deeplearning2/Kaggle/dogscats/data/'

In [29]:
ensamble = []

for i in range(5):
    model = train_one_learner()
    model.save(path + 'ensamble_{}_2.h5'.format(i))
    
    ensamble.append(model)

Training model for 4 epochs with learning rate 0.001
Training model for 8 epochs with learning rate 0.0001
Training model for 12 epochs with learning rate 1e-05


AttributeError: 'float' object has no attribute 'eval'

In [57]:
len(X_train)

23000

In [ ]:
def get_ens_pred(ensamble ,arr):
    ens_pred = []
    for model in ensamble:
        preds = model.predict(arr, batch_size=batch_size)
        ens_pred.append(preds)
    return np.stack(ens_pred).mean(axis=0)

In [ ]:
import keras.backend as K

def categorical_accuracy(y_true, y_pred):
    return K.cast(K.equal(K.argmax(y_true, axis=-1),
                          K.argmax(y_pred, axis=-1)),
                  K.floatx())

In [ ]:
import tensorflow as tf

val_avg_preds2 = get_ens_pred(ensamble, X_valid)

sess = tf.Session()
with sess.as_default():
    print(categorical_accuracy(y_valid, val_avg_preds2).eval().mean())